In [1]:
from pydrake.geometry import SceneGraph
from pydrake.geometry import StartMeshcat

In [2]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


Only registered “geometry sources” can introduce geometry into SceneGraph. Geometry sources will typically be other leaf systems, but, in the case of anchored (i.e., stationary) geometry, it could also be some other block of code (e.g., adding a common ground plane with which all systems’ geometries interact). For dynamic geometry (geometry whose pose depends on a Context), the geometry source must also provide pose/configuration values for all of the geometries the source owns, via a port connection on SceneGraph. For N geometry sources, the SceneGraph instance will have N pose/configuration input ports.

In [3]:
import numpy as np
from pydrake.common.containers import namedview
from pydrake.common.value import Value
from pydrake.math import RigidTransform, RotationMatrix
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import BasicVector, LeafSystem
from pydrake.trajectories import PiecewisePolynomial
from pydrake.all import LogVectorOutput

from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.framework import WitnessFunctionDirection
from pydrake.systems.framework import UnrestrictedUpdateEvent
from pydrake.systems.framework import EventStatus

from pydrake.systems.analysis import SimulatorConfig, ApplySimulatorConfig

from pydrake.geometry import FramePoseVector
from pydrake.geometry import GeometryFrame
from pydrake.geometry import GeometryInstance
from pydrake.geometry import Box, Capsule, Ellipsoid
from pydrake.geometry import Rgba
from pydrake.geometry import MeshcatVisualizer
from pydrake.geometry import MakePhongIllustrationProperties

from pydrake.math import RigidTransform

class HybridSystem(LeafSystem):
    def __init__(self):
        super().__init__()  # Don't forget to initialize the base class.

        self.name = "Example 2.2 from Impulsive and Hybrid Dynamical Systems"
        
        self.k1 = 1
        self.k2 = 0.1
        self.m1 = 2
        self.m2 = 1
        self.e = 0.8
        self.L = 0.5
        
        # Continuous State
        state_index = self.DeclareContinuousState(4)  
        self.DeclareStateOutputPort("y", state_index) 
        
        # https://drake.mit.edu/pydrake/pydrake.systems.framework.html?highlight=witnessfunction#pydrake.systems.framework.WitnessFunctio
        self.witness = self.MakeWitnessFunction(
            "witness", 
            WitnessFunctionDirection.kPositiveThenNonPositive, #kPositiveThenNonPositive, #kCrossesZero,
            self._guard,
            UnrestrictedUpdateEvent(self._reset)
        )
        
        
    def _guard(self, context):
        #t = context.get_time()
        x = context.get_continuous_state_vector()
        x1, x2, x3, x4 = x.GetAtIndex(0), x.GetAtIndex(1), x.GetAtIndex(2), x.GetAtIndex(3)
        L = self.L
        
        # See if their bumpers are touching...
        # easiest to just check absolute distance between cars...
        return L-(x1-x3)
        
        #return 666.0

    
    def _reset(self, context, event, state):
        x = context.get_continuous_state_vector()
        x1, x2, x3, x4 = x.GetAtIndex(0), x.GetAtIndex(1), x.GetAtIndex(2), x.GetAtIndex(3)
        m1,m2,k1,k2,e = self.m1, self.m2, self.k1, self.k2, self.e
        Δx1 = 0
        Δx2 = (-((1+e)*m2)/(m1+m2))*(x2-x4)
        Δx3 = 0
        Δx4 = (+((1+e)*m1)/(m1+m2))*(x2-x4)
        
        # We could just as well have returned just delx, but again, trying to make it explicit
        state.get_mutable_continuous_state().get_mutable_vector().SetFromVector(
            [x1+Δx1, 
             x2+Δx2,
             x3+Δx3,
             x4+Δx4]
        )
    
    def DoCalcTimeDerivatives(self, context, derivatives):
        x = context.get_continuous_state_vector()
        #x = context.get_mutable_continuous_state().get_mutable_vector()
        x1, x2, x3, x4 = x.GetAtIndex(0), x.GetAtIndex(1), x.GetAtIndex(2), x.GetAtIndex(3)

        x1dot = x2
        x2dot = -((self.k1 + self.k2)/self.m1)*x1 + (self.k2/self.m1)*x3
        x3dot = x4
        x4dot = (self.k2/self.m2)*x1 - (self.k2/self.m2)*x3
        
        derivatives.get_mutable_vector().SetAtIndex(0, x1dot)
        derivatives.get_mutable_vector().SetAtIndex(1, x2dot)
        derivatives.get_mutable_vector().SetAtIndex(2, x3dot)
        derivatives.get_mutable_vector().SetAtIndex(3, x4dot)

    # This override is required (exactly as typed) anytime
    # you use a Witness function. It also must take "context",
    # and, of course, "self"
    def DoGetWitnessFunctions(self, context):
        return [self.witness]
             

In [4]:
class HybridSystemGeometry(LeafSystem):
    
    def __init__(self, frame_id0, frame_id1, scene_graph):
        super().__init__()  # Don't forget to initialize the base class.
        self.x_port = self.DeclareVectorInputPort("state", 4)
        
        self.frame_id0 = frame_id0
        self.frame_id1 = frame_id1
        self.DeclareAbstractOutputPort("geometry_pose",
                                    alloc=lambda: Value(FramePoseVector()),
                                    calc=self.OutputGeometryPose)
        
        
            
    def OutputGeometryPose(self, context, framePoseVec):
        x = self.x_port.Eval(context)
        
        X1 = RigidTransform([x[0], 0, 0])
        X3 = RigidTransform([x[2], 0, 0])
        
        # Bizarre that we have to unpack like this...
        FPV = framePoseVec.get_value()
        
        FPV.set_value(self.frame_id0, X1)
        FPV.set_value(self.frame_id1, X3)    

In [5]:
hybrid_system = HybridSystem()

scene_graph = SceneGraph()
_id = scene_graph.RegisterSource("boxes")
fid0 = scene_graph.RegisterFrame(_id, GeometryFrame("f0"), )
fid1 = scene_graph.RegisterFrame(_id, GeometryFrame("f1"))

hybrid_system_geom = HybridSystemGeometry(fid0, fid1, scene_graph)

# Make some geometry, and set initial (geom) positions:
_box0 = GeometryInstance(RigidTransform([-0.5, 0, 0.5]), Box(1, 1, 1), "box_instance0")
_box1 = GeometryInstance(RigidTransform([0.5 + hybrid_system.L, 0, 0.5]), Box(1, 1, 1), "box_instance1")

scene_graph.AssignRole(_id, scene_graph.RegisterGeometry(_id, fid0, _box0), MakePhongIllustrationProperties(np.array([.9, .1, 0, 1]).T))
scene_graph.AssignRole(_id, scene_graph.RegisterGeometry(_id, fid1, _box1), MakePhongIllustrationProperties(np.array([.9, .9, 0, 1]).T))

builder = DiagramBuilder()
builder.AddSystem(scene_graph)
builder.AddSystem(hybrid_system)
builder.AddSystem(hybrid_system_geom)

# Connect system output to geom_input, and geom_ouput to scenegraph pose input.
builder.Connect(hybrid_system.GetOutputPort('y'),  hybrid_system_geom.get_input_port())
builder.Connect(hybrid_system_geom.GetOutputPort("geometry_pose"),  scene_graph.get_source_pose_port(_id))

# Convenience function to add meshcat to builder
MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

diagram = builder.Build()

# Display Meshcat in Notebook:

In [6]:
from IPython.display import IFrame
address = meshcat.web_url()
print("Or, just view it here in the browser...")
display(IFrame(src=address, width='100%', height='800px'))

Or, just view it here in the browser...


In [7]:
ics = [0.32, -0.8, 0.01, 0.6]

simulator = Simulator(diagram)
sim_context = simulator.get_mutable_context()
sim_context.SetTime(0)
sim_context.SetContinuousState(ics)
sim_context.SetAccuracy(1e-10)      #<--- hybrid model needs this.

simulator.set_target_realtime_rate(4.0)

#print("Use the slider in the MeshCat controls to apply elbow torque.")
print("Press 'Stop Simulation' in MeshCat to continue.")
meshcat.AddButton("Stop Simulation")
while meshcat.GetButtonClicks("Stop Simulation") < 1:
    simulator.AdvanceTo(simulator.get_context().get_time() + 0.025)

meshcat.DeleteAddedControls()

Press 'Stop Simulation' in MeshCat to continue.
